In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.core.display import HTML
from IPython.display import display
import torch
from tqdm import tqdm
from pathlib import Path

from torch.utils.data import TensorDataset

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline